In [2]:
# https://drive.google.com/file/d/1OxmGA6YxquGzoNL3erb6exgGWjeWp5Lb/view?usp=sharing
!gdown --id 1OxmGA6YxquGzoNL3erb6exgGWjeWp5Lb

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1OxmGA6YxquGzoNL3erb6exgGWjeWp5Lb
From (redirected): https://drive.google.com/uc?id=1OxmGA6YxquGzoNL3erb6exgGWjeWp5Lb&confirm=t&uuid=f21555d3-ab4c-423a-af53-d04869ec51a0
To: /content/assignment.zip
100% 108M/108M [00:04<00:00, 26.7MB/s]


## Data Prepocessing

In [3]:
!unzip -q assignment.zip

In [4]:
train_dir = 'assignment/train'
test_dir = 'assignment/test'

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)



In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 13493 images belonging to 33 classes.
Found 3361 images belonging to 33 classes.


Let's use ResNet50

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [13]:
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers[-50:]:
    layer.trainable = False

In [14]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 25,719,713 (98.11 MB)

 Trainable params: 2,132,001 (8.13 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [15]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(train_generator,
    epochs=10,
    validation_data=validation_generator)

Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.1004 - loss: 3.2974

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


422/422 ━━━━━━━━━━━━━━━━━━━━ 225s 505ms/step - accuracy: 0.1005 - loss: 3.2969 - val_accuracy: 0.2068 - val_loss: 2.6826
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 253s 491ms/step - accuracy: 0.2401 - loss: 2.5712 - val_accuracy: 0.3228 - val_loss: 2.2639
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 263s 494ms/step - accuracy: 0.3298 - loss: 2.2086 - val_accuracy: 0.3567 - val_loss: 2.0657
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 206s 480ms/step - accuracy: 0.3841 - loss: 1.9690 - val_accuracy: 0.3779 - val_loss: 1.9412
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 262s 482ms/step - accuracy: 0.4405 - loss: 1.7822 - val_accuracy: 0.4832 - val_loss: 1.6559
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 208s 487ms/step - accuracy: 0.5001 - loss: 1.6368 - val_accuracy: 0.5070 - val_loss: 1.5454
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 204s 478ms/step - accuracy: 0.5230 - loss: 1.5342 - val_accuracy: 0.5632 - val_loss: 1.4291
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 246s 577ms/step - accuracy: 0.5744 - loss: 1.38

In [21]:
model.fit(train_generator,
    epochs=5,
    validation_data=validation_generator)

Epoch 1/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 207s 484ms/step - accuracy: 0.6392 - loss: 1.1596 - val_accuracy: 0.6629 - val_loss: 1.1015
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 210s 492ms/step - accuracy: 0.6407 - loss: 1.1272 - val_accuracy: 0.6825 - val_loss: 1.0304
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 204s 479ms/step - accuracy: 0.6674 - loss: 1.0516 - val_accuracy: 0.7051 - val_loss: 0.9800
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 206s 481ms/step - accuracy: 0.6926 - loss: 0.9927 - val_accuracy: 0.7123 - val_loss: 0.9313
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 264s 486ms/step - accuracy: 0.6994 - loss: 0.9511 - val_accuracy: 0.7093 - val_loss: 0.8934


In [22]:
# Data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    classes = None
)

# Predict labels for the test set
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=-1)


Found 5641 images belonging to 1 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5641/5641 ━━━━━━━━━━━━━━━━━━━━ 44s 7ms/step


In [24]:
import pandas as pd

# Get filenames and remove extensions
filenames = test_generator.filenames
ids = [filename.split('/')[-1].split('.')[0] for filename in filenames]

# Create DataFrame
submission_df = pd.DataFrame({'id': ids, 'label': predicted_classes})

# Save to CSV
submission_df.to_csv('submission.csv', index=False)


In [25]:
class_indices = train_generator.class_indices
class_indices

{'Apple Braeburn': 0,
 'Apple Granny Smith': 1,
 'Apricot': 2,
 'Avocado': 3,
 'Banana': 4,
 'Blueberry': 5,
 'Cactus fruit': 6,
 'Cantaloupe': 7,
 'Cherry': 8,
 'Clementine': 9,
 'Corn': 10,
 'Cucumber Ripe': 11,
 'Grape Blue': 12,
 'Kiwi': 13,
 'Lemon': 14,
 'Limes': 15,
 'Mango': 16,
 'Onion White': 17,
 'Orange': 18,
 'Papaya': 19,
 'Passion Fruit': 20,
 'Peach': 21,
 'Pear': 22,
 'Pepper Green': 23,
 'Pepper Red': 24,
 'Pineapple': 25,
 'Plum': 26,
 'Pomegranate': 27,
 'Potato Red': 28,
 'Raspberry': 29,
 'Strawberry': 30,
 'Tomato': 31,
 'Watermelon': 32}

In [26]:
submission_df["Names"] = submission_df["label"].map(class_indices)

In [27]:
submission_df.head()

,id,label,Names
0,0000,27,NaN
1,0001,12,NaN
2,0002,16,NaN
3,0003,12,NaN
4,0004,16,NaN


In [29]:
model.save("model.keras")